#### Import all libraries

In [102]:
from __future__ import print_function
import os
import collections
import SimpleITK as sitk
import numpy as np
import six
import radiomics
from radiomics import firstorder, glcm, imageoperations, shape, glrlm, glszm, getTestCase
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
from pathlib import Path
import sys
import logging as logger
import time
import json
import os
import logging.config
import pandas as pd
import gc
import math

In [103]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [104]:
#Read the entire Benthiemer sandstone
global image_1, label_1
#change the path to the correct directory before running the code
# please note: the dimensions of the image is 800 x 800 x 800
image_1 = sitk.ReadImage('D:\\Ankita PhD\\pyradiomics\\data\\experiments\\data18.nrrd')

#### Extraction Settings

In [105]:
settings = {}
settings['label'] = 255
settings['binWidth'] = 1
settings['padDistance'] = 0
settings['distances'] = [1]

#### Log messages

In [66]:
log_file = 'D:\\Ankita PhD\\pyradiomics\\log_messages\\data14_sandpack.txt'
handler = logging.FileHandler(filename=log_file, mode='w')  # overwrites log_files from previous runs. Change mode to 'a' to append.
formatter = logging.Formatter("%(levelname)s:%(name)s: %(message)s")  # format string for log messages
handler.setFormatter(formatter)
radiomics.logger.addHandler(handler)

# Control the amount of logging stored by setting the level of the logger. N.B. if the level is higher than the
# Verbositiy level, the logger level will also determine the amount of information printed to the output
radiomics.logger.setLevel(logging.DEBUG)

In [106]:
%%time
# the size of the mask should be the same as that of the image
mask_x = 200
mask_y = 200
mask_z = 200

v1 = []
v2 = []
v3 = []
v4 = []
v5 = []
v6 = []
v7 = []
v8 = []
v9 = []
v10 = []
v11 = []
v12 = []
v13 = []
v14 = []
v15 = []
v16 = []

# here the window_size refers to the region of mask that will be used to segment the ROI from the image. The window
# sizes increase with every iteration
u_window_size = [200] #10, 20, 30, 40, 50, 60, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700, 750, 800]
global window_size, total_iter, expect_iter, i_loc, j_loc, z_loc, bb, correctedMask, croppedImage, croppedMask, glcm_window
for window_size in u_window_size:  
    
    i_loc = int((mask_x - window_size)/2)
    j_loc = int((mask_y - window_size)/2)
    k_loc = int((mask_z - window_size)/2)
    
    # Generate the mask
    full_mask = np.zeros((mask_x, mask_y, mask_z), dtype = np.uint8)
    full_mask[i_loc:i_loc+window_size,j_loc:j_loc+window_size,k_loc:k_loc+window_size] = 255
    #print("Window Location", str(i_loc), str(i_loc+window_size), str(j_loc), str(j_loc+window_size), str(k_loc), str(k_loc+window_size))
    label_1 = sitk.GetImageFromArray(full_mask, isVector=False)
    del full_mask

    bb, correctedMask = imageoperations.checkMask(image_1, label_1, label=255)
    if correctedMask is not None:
        label_1 = correctedMask
    croppedImage, croppedMask = imageoperations.cropToTumorMask(image_1, label_1, bb)
    del correctedMask, label_1

    #calculate secondorder stats
    glszmFeatures = glszm.RadiomicsGLSZM(croppedImage, croppedMask, **settings)
    glszmFeatures.enableAllFeatures()
    print('Calculating GLSZM features...',)
    result = glszmFeatures.execute()
    print('done')
    
    print('Calculated GLSZM features: ')
    for key, value in six.iteritems(result):
        print('\t', key, ':', value)
        
    print("Completed Domain Calculations for Window ", str(window_size))

Calculating GLSZM features...
done
Calculated GLSZM features: 
	 GrayLevelNonUniformity : 33640.325074463086
	 GrayLevelNonUniformityNormalized : 0.007218391587621626
	 GrayLevelVariance : 2570.893724948612
	 HighGrayLevelZoneEmphasis : 16799.474488789823
	 LargeAreaEmphasis : 9.730984045663396
	 LargeAreaHighGrayLevelEmphasis : 244016.891885675
	 LargeAreaLowGrayLevelEmphasis : 0.001794604743569884
	 LowGrayLevelZoneEmphasis : 0.00019445907171714575
	 SizeZoneNonUniformity : 2618816.309077855
	 SizeZoneNonUniformityNormalized : 0.5619339757606554
	 SmallAreaEmphasis : 0.7776104377993489
	 SmallAreaHighGrayLevelEmphasis : 12439.729136341379
	 SmallAreaLowGrayLevelEmphasis : 0.00014307799421306073
	 ZoneEntropy : 8.630407417688525
	 ZonePercentage : 0.582545375
	 ZoneVariance : 6.7842531080207245
Completed Domain Calculations for Window  200
Wall time: 3.53 s


In [107]:
im_exp = sitk.GetArrayFromImage(croppedImage)
im_exp

array([[[ 23,  21,  24, ...,  58,  61,  62],
        [ 26,  28,  25, ...,  90, 100,  99],
        [ 26,  24,  20, ..., 134, 142, 148],
        ...,
        [179, 166, 167, ..., 146, 141, 148],
        [179, 173, 168, ..., 145, 137, 134],
        [173, 179, 169, ..., 137, 122, 117]],

       [[ 16,  19,  22, ...,  55,  55,  59],
        [ 21,  26,  26, ...,  89,  93,  94],
        [ 20,  25,  25, ..., 135, 142, 142],
        ...,
        [180, 166, 162, ..., 152, 148, 158],
        [180, 172, 164, ..., 144, 146, 151],
        [174, 173, 171, ..., 131, 132, 136]],

       [[ 16,  25,  19, ...,  53,  50,  57],
        [ 18,  21,  24, ...,  88,  89,  95],
        [ 16,  19,  23, ..., 133, 140, 145],
        ...,
        [177, 169, 161, ..., 157, 160, 161],
        [182, 177, 164, ..., 147, 154, 158],
        [172, 171, 167, ..., 136, 139, 149]],

       ...,

       [[162, 173, 179, ..., 181, 181, 172],
        [170, 171, 174, ..., 175, 176, 180],
        [171, 177, 179, ..., 179, 182, 177

In [108]:
glszm_3 = glszmFeatures.P_glszm
glszm_3

array([[[0., 1., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [4., 0., 0., ..., 0., 0., 0.],
        ...,
        [5., 0., 0., ..., 0., 0., 0.],
        [7., 0., 0., ..., 0., 0., 0.],
        [2., 1., 1., ..., 0., 0., 0.]]])

In [109]:
glszm_3.shape

(1, 253, 157)

In [110]:
type(glszm_3)

numpy.ndarray

In [111]:
glszm = glszm_3[0, 0:253, 0:157]
glszm.shape

(253, 157)

In [126]:
pd.DataFrame(glszm).to_csv("D:\\Ankita PhD\\pyradiomics\\data\\data18_glszm.csv")

In [112]:
#The graylevels in ROI after discretization
graylevels = glszmFeatures.coefficients['grayLevels']
graylevels

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  65,  66,
        67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,
        80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,
        93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104, 105,
       106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118,
       119, 120, 121, 122, 123, 124, 125, 126, 127, 129, 130, 131, 132,
       133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145,
       146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158,
       159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171,
       172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 18

In [113]:
# sum of all the size zones for all the graylevels of a particular size zones
# for every column of GLSZM, add all the rows
pss = glszmFeatures.coefficients['ps']
pss

array([[3.414225e+06, 6.893820e+05, 2.334920e+05, 1.073700e+05,
        5.984700e+04, 3.728200e+04, 2.533800e+04, 1.803900e+04,
        1.328600e+04, 1.004900e+04, 7.777000e+03, 6.450000e+03,
        5.183000e+03, 4.106000e+03, 3.393000e+03, 2.955000e+03,
        2.464000e+03, 2.157000e+03, 1.698000e+03, 1.585000e+03,
        1.366000e+03, 1.230000e+03, 1.091000e+03, 1.003000e+03,
        8.330000e+02, 7.550000e+02, 6.040000e+02, 6.270000e+02,
        5.280000e+02, 4.640000e+02, 3.990000e+02, 3.620000e+02,
        3.580000e+02, 3.040000e+02, 3.060000e+02, 2.880000e+02,
        2.320000e+02, 2.100000e+02, 2.070000e+02, 2.090000e+02,
        1.620000e+02, 1.500000e+02, 1.610000e+02, 1.410000e+02,
        1.350000e+02, 1.180000e+02, 1.230000e+02, 1.190000e+02,
        1.020000e+02, 1.000000e+02, 8.300000e+01, 7.900000e+01,
        6.700000e+01, 6.700000e+01, 6.700000e+01, 6.900000e+01,
        5.900000e+01, 5.200000e+01, 6.000000e+01, 5.100000e+01,
        3.400000e+01, 4.700000e+01, 3.00

In [127]:
pss.shape

(1, 157)

In [114]:
#Np is the number of voxels in the image
Npp = glszmFeatures.coefficients['Np']
Npp

array([8000000.])

In [115]:
#The total number of size zones in the GLSZM (add all the elements of P_glszm)
Nzz = glszmFeatures.coefficients['Nz']
Nzz

array([4660363.])

In [116]:
#Number of all possible discreet intensity values in the image: This incluedes  NgVector = range(1, Ng + 1) i.e.
#all the missing grayvalues as well. Later the missing grayvalues are excluded. 
Ngg = glszmFeatures.coefficients['Ng']
Ngg

256

In [117]:
#sum all the size zones for a particular graylevel
#for every row of GLSZM add the columns
pgg = glszmFeatures.coefficients['pg']
pgg

array([[1.0000e+00, 1.0000e+00, 4.0000e+00, 3.0000e+00, 5.0000e+00,
        1.5000e+01, 1.1000e+01, 2.7000e+01, 4.5000e+01, 7.1000e+01,
        1.3800e+02, 2.2200e+02, 3.3400e+02, 4.7800e+02, 7.5000e+02,
        1.0880e+03, 1.5430e+03, 2.1760e+03, 2.8950e+03, 3.8260e+03,
        4.8460e+03, 5.9930e+03, 7.4080e+03, 8.7150e+03, 1.0371e+04,
        1.1915e+04, 1.3735e+04, 1.5190e+04, 1.6800e+04, 1.8689e+04,
        2.0139e+04, 2.1814e+04, 2.3037e+04, 2.4582e+04, 2.5617e+04,
        2.6827e+04, 2.7575e+04, 2.8177e+04, 2.8889e+04, 2.9030e+04,
        2.9058e+04, 2.9043e+04, 2.8975e+04, 2.8705e+04, 2.8527e+04,
        2.8242e+04, 2.8035e+04, 2.7713e+04, 2.7129e+04, 2.6855e+04,
        2.6339e+04, 2.5760e+04, 2.5692e+04, 2.5238e+04, 2.5197e+04,
        2.4769e+04, 2.4376e+04, 2.4109e+04, 2.3865e+04, 2.3588e+04,
        2.3171e+04, 2.3164e+04, 2.2813e+04, 2.2267e+04, 2.2375e+04,
        2.2228e+04, 2.1849e+04, 2.1539e+04, 2.1454e+04, 2.1333e+04,
        2.1248e+04, 2.0920e+04, 2.1204e+04, 2.06

In [118]:
pgg.shape

(1, 253)

In [119]:
ivv = glszmFeatures.coefficients['ivector']
ivv

array([  1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,  11.,
        12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,  22.,
        23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,  33.,
        34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,  44.,
        45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,  55.,
        56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  65.,  66.,  67.,
        68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,  77.,  78.,
        79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,  88.,  89.,
        90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,  99., 100.,
       101., 102., 103., 104., 105., 106., 107., 108., 109., 110., 111.,
       112., 113., 114., 115., 116., 117., 118., 119., 120., 121., 122.,
       123., 124., 125., 126., 127., 129., 130., 131., 132., 133., 134.,
       135., 136., 137., 138., 139., 140., 141., 142., 143., 144., 145.,
       146., 147., 148., 149., 150., 151., 152., 15

In [120]:
ivv.shape

(253,)

In [128]:
jvv = glszmFeatures.coefficients['jvector']
jvv

array([  1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,  11.,
        12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,  22.,
        23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,  33.,
        34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,  44.,
        45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,  55.,
        56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,  66.,
        67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,  77.,
        78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,  88.,
        89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,  99.,
       100., 101., 102., 103., 104., 105., 106., 107., 108., 109., 110.,
       111., 112., 113., 114., 115., 116., 117., 118., 119., 120., 121.,
       122., 123., 124., 126., 128., 129., 130., 132., 133., 138., 139.,
       140., 143., 144., 145., 146., 148., 149., 150., 152., 154., 156.,
       157., 159., 161., 164., 165., 166., 170., 17

In [122]:
jvv.shape

(157,)

In [123]:
hilae = np.sum(glszm_3 * (ivv[None, :, None] ** 2) * (jvv[None, None, :] ** 2), (1, 2)) / Nzz
hilae

array([244016.89188567])

In [124]:
pd.DataFrame(jvv).to_csv("D:\\Ankita PhD\\pyradiomics\\data\\data18_jvv.csv")

In [125]:
pd.DataFrame(ivv).to_csv("D:\\Ankita PhD\\pyradiomics\\data\\data18_ivv.csv")